In [6]:
import pandas as pd
import numpy as np
import geopandas as gpd
import torch
import torch.nn.functional as F
import networkx as nx
import community as community_louvain 
from sklearn.preprocessing import MinMaxScaler
from torch_geometric.data import HeteroData
from shapely.ops import transform
from src.utils import perform_semantic_sjoin, normalize_features
TARGET_CRS = "EPSG:2154" 
SCALER = MinMaxScaler()

In [2]:
df_adr =  pd.read_csv('data/adresses-92.csv', delimiter=';')
gdf_adr = gpd.GeoDataFrame(
    df_adr, 
    geometry=gpd.points_from_xy(df_adr['lon'], df_adr['lat']), 
    crs="EPSG:4326"
)
gdf_bat = gpd.read_file('data/BDT_3-5_GPKG_LAMB93_D092-ED2025-06-15.gpkg', layer='batiment')
gdf_par = gpd.read_file('data/cadastre-92-parcelles.json')


# 1.2 Normalisation des CRS (Lambert 93)
gdf_adr = gdf_adr.to_crs(TARGET_CRS)
gdf_bat = gdf_bat.to_crs(TARGET_CRS)
gdf_par = gdf_par.to_crs(TARGET_CRS)

# Assurer que les colonnes ID existent et sont renommées
gdf_adr = gdf_adr.rename(columns={'id': 'id_adr'}).assign(id_adr=lambda x: x['id_adr'].astype(str))

# Bâtiments : Créer ID et features numériques (hauteur/surface)
gdf_bat = gdf_bat.reset_index().rename(columns={'cleabs': 'id_bat_idx'}).assign(
    id_bat=lambda x: 'BAT_' + x['id_bat_idx'].astype(str),
    # Utiliser l'aire réelle comme proxy de surface
    surface=lambda x: x.force_2d().area
).drop(columns=['id_bat_idx'])

# Parcelles : Forcer la superficie à être numérique et gérer les ID
gdf_par = gdf_par.rename(columns={'id': 'id_par', 'superficie': 'superficie'}).assign(
    id_par=lambda x: x['id_par'].astype(str),
    superficie=lambda x: pd.to_numeric(x.geometry.area, errors='coerce').fillna(0.0).astype(np.float32)
)

gdf_parf = gdf_par[['id_par','superficie','geometry']].copy()
gdf_batf = gdf_bat[['id_bat','surface','hauteur','geometry']].copy()
gdf_adrf = gdf_adr[['id_adr','x','y','geometry']].copy()

/tmp/ipykernel_4005/4051318219.py:1: DtypeWarning: Columns (16,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df_adr =  pd.read_csv('data/adresses-92.csv', delimiter=';')


In [3]:
doc_urba = gpd.read_file("data/wfs_du.gpkg",layer='zone_urba')
doc_urba = doc_urba[['gid','partition','libelle','typezone','geometry']].copy()
doc_urba = doc_urba.set_crs(4326, allow_override=True)

In [4]:
gdf_parf = perform_semantic_sjoin(gdf_parf, doc_urba)

Reprojection des Zones PLU vers EPSG:2154...
Calcul du point représentatif des Parcelles (point garanti dans la géométrie)...
Exécution de la jointure spatiale (Parcelle.ReprensentativePoint WITHIN Zone PLU)...


In [8]:
# Imputation des NaN PLU (Valeurs manquantes = 'HORS_PLU' pour le GNN)
gdf_parf['LIBELLE'] = gdf_parf['LIBELLE'].fillna('HORS_PLU')
gdf_parf['TYPEZONE'] = gdf_parf['TYPEZONE'].fillna('HORS_PLU')

# Encodage One-Hot des labels (features sémantiques)
df_plu_encoded = pd.get_dummies(gdf_parf[['LIBELLE', 'TYPEZONE']], prefix=['PLU_LIBELLE', 'TYPEZONE'])

# APPLICATION DE LA NORMALISATION Min-Max (Critique pour la performance)
gdf_adrf['x'] = gdf_adr.geometry.x
gdf_adrf['y'] = gdf_adr.geometry.y

gdf_adrf = normalize_features(gdf_adrf, ['x', 'y']) 
gdf_batf = normalize_features(gdf_batf, ['hauteur', 'surface'])
gdf_parf = normalize_features(gdf_parf, ['superficie'])

In [17]:
# Préparation des tenseurs de features NORMALISÉS
par_features_base = gdf_parf[['superficie']].values
par_features = np.hstack([par_features_base, df_plu_encoded.values])

# --- 1.4. Création des Relations (Arêtes) ---

# A) Relation Bâtiment -> Parcelle (Lien pondéré)
sjoin_bp_id = gpd.sjoin(gdf_batf[['id_bat', 'geometry']], gdf_parf[['id_par', 'geometry']], how='left', predicate='intersects', lsuffix='bat', rsuffix='par')
edge_index_bp_df = sjoin_bp_id[['id_bat', 'id_par']].dropna().reset_index(drop=True)

# Poids de l'arête B->P : SIMULATION (1.0) en attendant le calcul réel d'intersection
edge_index_bp_df['intersection_perc'] = 1.0 

# B) Relation Adresse -> Bâtiment (Jointure de Proximité)
print("Calcul de la relation Adresse -> Bâtiment (sjoin_nearest)...")

# Utilisation de sjoin_nearest pour gérer les adresses sur la voie publique (max 100m)
MAX_DISTANCE_METERS = 100 
sjoin_ab = gpd.sjoin_nearest(
    gdf_adrf, 
    gdf_batf[['id_bat', 'geometry']], 
    how='left', 
    max_distance=MAX_DISTANCE_METERS, 
    lsuffix='adr', 
    rsuffix='bat'
)
edge_index_ab_df = sjoin_ab[['id_adr', 'id_bat']].dropna().reset_index(drop=True)

Calcul de la relation Adresse -> Bâtiment (sjoin_nearest)...


In [18]:
# --- 1.5. Conversion Finale pour PyTorch ---

# Création des Mappings ID réel -> Index numérique
adr_map = {id: i for i, id in enumerate(gdf_adrf['id_adr'].unique())}
bat_map = {id: i for i, id in enumerate(gdf_batf['id_bat'].unique())}
par_map = {id: i for i, id in enumerate(gdf_parf['id_par'].unique())}

# Conversion des DataFrames d'arêtes en indices numériques
edge_index_bp_df['bat_src_idx'] = edge_index_bp_df['id_bat'].map(bat_map)
edge_index_bp_df['par_dst_idx'] = edge_index_bp_df['id_par'].map(par_map)
edge_index_ab_df['adr_src_idx'] = edge_index_ab_df['id_adr'].map(adr_map)
edge_index_ab_df['bat_dst_idx'] = edge_index_ab_df['id_bat'].map(bat_map)

# Tenseurs
edge_index_bp = torch.tensor(edge_index_bp_df[['bat_src_idx', 'par_dst_idx']].values.T.astype(np.int64), dtype=torch.long)
edge_attr_bp = torch.tensor(edge_index_bp_df[['intersection_perc']].values, dtype=torch.float)
edge_index_ab = torch.tensor(edge_index_ab_df[['adr_src_idx', 'bat_dst_idx']].values.T.astype(np.int64), dtype=torch.long)

# Tenseurs de Features (normalisés)
adr_x = torch.tensor(gdf_adr[['x', 'y']].values.astype(np.float32), dtype=torch.float)
bat_x = torch.tensor(gdf_bat[['hauteur', 'surface']].values.astype(np.float32), dtype=torch.float)
par_x = torch.tensor(par_features, dtype=torch.float)